In [6]:
!pip install pandas-datareader
import time
import numpy as np
import pandas as pd
import json
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import bs4 as bs
import pickle
import requests
from collections import OrderedDict
import pytz


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/bin/python -m pip install --upgrade pip' command.


In [7]:
def get_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker[:-1]
        tickers.append(ticker)

    
    tickers.sort()
    tickers.remove('BF.B')
    tickers.remove('BRK.B')
    tickers.remove('CARR')
    tickers.remove('DPZ')
    tickers.remove('DXCM')
    tickers.remove('OTIS')
    tickers.remove('WST')
    
    return tickers

def save_sp500_tickers(tickers):
    with open('sp500tickers.pickle','wb') as f:
            pickle.dump(tickers, f)

            
def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.025
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0

In [8]:
tickers = get_sp500_tickers()

In [16]:
data_indicators = OrderedDict()
stock_data_indicators = pd.DataFrame()
hm_days = 7
for ticker in tickers:
    df = pd.read_csv("stocks_dfs/{}.csv".format(ticker), index_col=0, parse_dates=['Date'])
    df = df[['Adj Close','Volume']]
    df.fillna(0, inplace=True)
    for i in range(1, hm_days+1):
        df['{}d'.format(i)] = (df['Adj Close'].shift(-i) - df['Adj Close']) / df['Adj Close']
        
    df['50MA'] = df['Adj Close'].rolling(50).mean()
    df['25MA'] = df['Adj Close'].rolling(25).mean()
    df['10MA'] = df['Adj Close'].rolling(10).mean()
    df['5MA'] = df['Adj Close'].rolling(5).mean()

    df['50STD'] = df['Adj Close'].rolling(50).std()
    df['25STD'] = df['Adj Close'].rolling(25).std()
    df['10STD'] = df['Adj Close'].rolling(10).std()

    df['50UBB'] = df['50MA'] + (df['50STD'] * 2)
    df['25UBB'] = df['25MA'] + (df['25STD'] * 2)

    df['50LBB'] = df['50MA'] - (df['50STD'] * 2)
    df['25LBB'] = df['25MA'] - (df['25STD'] * 2)
    
    df['RSI5'] = get_rsi(5,df)
    df['RSI10'] = get_rsi(10,df)
    df['RSI25'] = get_rsi(25,df)
    df['RSI50'] = get_rsi(50,df)
    
    df['MACD5-10'] = get_macd(5,10,df)
    df['MACD5-25'] = get_macd(5,25,df)
    df['MACD5-50'] = get_macd(5,50,df)

    df['MACD10-25'] = get_macd(10,25,df)
    df['MACD10-50'] = get_macd(10,50,df)

    df['MACD25-50'] = get_macd(25,50,df)
    
    df['5DMomentum'] = get_n_day_momentum(5,df)
    df['10DMomentum'] = get_n_day_momentum(10,df)
    df['25DMomentum'] = get_n_day_momentum(25,df)
    df['50DMomentum'] = get_n_day_momentum(50,df)


    
    data_indicators[ticker] = df
    

stock_data_indicators = pd.concat(data_indicators.values(),keys=tickers,names=['Ticker','Date'])

stock_data_indicators = stock_data_indicators.swaplevel()
stock_data_indicators = stock_data_indicators.sort_index()

In [17]:
stock_data_indicators

Adj Close       Volume        1d        2d        3d  \
Date       Ticker                                                          
2010-01-04 A        20.436504    3815500.0 -0.010862 -0.014377 -0.015655   
           AAL       4.496876    9837300.0  0.113208  0.067086  0.098533   
           AAP      39.293575    1701700.0 -0.005943  0.002724  0.002476   
           AAPL     26.538483  123432400.0  0.001729 -0.014205 -0.016027   
           ABC      22.813559    2455900.0 -0.007134 -0.016522 -0.032294   
...                       ...          ...       ...       ...       ...   
2019-12-31 YUM     100.280571    1271000.0       NaN       NaN       NaN   
           ZBH     149.308990     612400.0       NaN       NaN       NaN   
           ZBRA    255.440002     363500.0       NaN       NaN       NaN   
           ZION     51.538563    1246900.0       NaN       NaN       NaN   
           ZTS     131.945450    1128600.0       NaN       NaN       NaN   

                         4d        5d        6d        7d        50MA  ...  \
Date       Ticker                                                      ...   
2010-01-04 A      -0.015975 -0.015336 -0.027157 -0.019489         NaN  ...   
           AAL     0.077568  0.056604  0.064990  0.148847         NaN  ...   
           AAP     0.006439 -0.003467 -0.020802 -0.007182         NaN  ...   
           AAPL   -0.009485 -0.018223 -0.029391 -0.015700         NaN  ...   
           ABC    -0.021780 -0.010890 -0.004131  0.015396         NaN  ...   
...                     ...       ...       ...       ...         ...  ...   
2019-12-31 YUM          NaN       NaN       NaN       NaN  100.330244  ...   
           ZBH          NaN       NaN       NaN       NaN  143.649495  ...   
           ZBRA         NaN       NaN       NaN       NaN  243.789800  ...   
           ZION         NaN       NaN       NaN       NaN   49.479128  ...   
           ZTS          NaN       NaN       NaN       NaN  123.233558  ...   

                   MACD5-10  MACD5-25   MACD5-50  MACD10-25  MACD10-50  \
Date       Ticker                                                        
2010-01-04 A       0.000000  0.000000   0.000000   0.000000   0.000000   
           AAL     0.000000  0.000000   0.000000   0.000000   0.000000   
           AAP     0.000000  0.000000   0.000000   0.000000   0.000000   
           AAPL    0.000000  0.000000   0.000000   0.000000   0.000000   
           ABC     0.000000  0.000000   0.000000   0.000000   0.000000   
...                     ...       ...        ...        ...        ...   
2019-12-31 YUM     0.270127  0.477926  -1.030058   0.207799  -1.300185   
           ZBH     0.257086  1.993965   4.850174   1.736880   4.593088   
           ZBRA    0.183174  3.316510  12.101816   3.133336  11.918642   
           ZION    0.221980  0.958481   2.252459   0.736501   2.030479   
           ZTS     1.415003  4.927581   7.074713   3.512579   5.659711   

                   MACD25-50  5DMomentum  10DMomentum  25DMomentum  \
Date       Ticker                                                    
2010-01-04 A        0.000000         NaN          NaN          NaN   
           AAL      0.000000         NaN          NaN          NaN   
           AAP      0.000000         NaN          NaN          NaN   
           AAPL     0.000000         NaN          NaN          NaN   
           ABC      0.000000         NaN          NaN          NaN   
...                      ...         ...          ...          ...   
2019-12-31 YUM     -1.507984    0.915901    -0.159279     3.096123   
           ZBH      2.856208   -1.316162    -0.081192     6.073685   
           ZBRA     8.785306   -1.290009    -0.009995    11.869995   
           ZION     1.293979    0.575737     0.724636     1.945599   
           ZTS      2.147132   -0.019928     5.872002    12.501678   

                   50DMomentum  
Date       Ticker               
2010-01-04 A               NaN  
           AAL             NaN  
          

In [18]:
stock_data_indicators['target'] = list(map( buy_sell_hold,
                                               stock_data_indicators['1d'.format(ticker)],
                                               stock_data_indicators['2d'.format(ticker)],
                                               stock_data_indicators['3d'.format(ticker)],
                                               stock_data_indicators['4d'.format(ticker)],
                                               stock_data_indicators['5d'.format(ticker)],
                                               stock_data_indicators['6d'.format(ticker)],
                                               stock_data_indicators['7d'.format(ticker)] ))

In [19]:
stock_data_indicators.head()

Adj Close       Volume        1d        2d        3d  \
Date       Ticker                                                         
2010-01-04 A       20.436504    3815500.0 -0.010862 -0.014377 -0.015655   
           AAL      4.496876    9837300.0  0.113208  0.067086  0.098533   
           AAP     39.293575    1701700.0 -0.005943  0.002724  0.002476   
           AAPL    26.538483  123432400.0  0.001729 -0.014205 -0.016027   
           ABC     22.813559    2455900.0 -0.007134 -0.016522 -0.032294   

                         4d        5d        6d        7d  50MA  ...  \
Date       Ticker                                                ...   
2010-01-04 A      -0.015975 -0.015336 -0.027157 -0.019489   NaN  ...   
           AAL     0.077568  0.056604  0.064990  0.148847   NaN  ...   
           AAP     0.006439 -0.003467 -0.020802 -0.007182   NaN  ...   
           AAPL   -0.009485 -0.018223 -0.029391 -0.015700   NaN  ...   
           ABC    -0.021780 -0.010890 -0.004131  0.015396   NaN  ...   

                   MACD5-25  MACD5-50  MACD10-25  MACD10-50  MACD25-50  \
Date       Ticker                                                        
2010-01-04 A            0.0       0.0        0.0        0.0        0.0   
           AAL          0.0       0.0        0.0        0.0        0.0   
           AAP          0.0       0.0        0.0        0.0        0.0   
           AAPL         0.0       0.0        0.0        0.0        0.0   
           ABC          0.0       0.0        0.0        0.0        0.0   

                   5DMomentum  10DMomentum  25DMomentum  50DMomentum  target  
Date       Ticker                                                             
2010-01-04 A              NaN          NaN          NaN          NaN      -1  
           AAL            NaN          NaN          NaN          NaN       1  
           AAP            NaN          NaN          NaN          NaN       0  
           AAPL           NaN          NaN          NaN          NaN      -1  
           ABC            NaN          NaN          NaN          NaN      -1  

[5 rows x 35 columns]

In [20]:
stock_data_indicators.drop(columns=['1d', '2d','3d','4d','5d','6d','7d'], inplace=True)
stock_data_indicators.replace([np.inf, -np.inf], np.nan, inplace = True)
stock_data_indicators.fillna(0, inplace=True)
stock_data_indicators = stock_data_indicators.loc[(slice('2010-03-16','2019-12-31')),:]
stock_data_indicators.head()

Adj Close       Volume       50MA       25MA       10MA  \
Date       Ticker                                                            
2010-03-16 A       21.820704    3285900.0  20.141775  20.705770  21.645719   
           AAL      7.325101    5915400.0   5.946059   6.822809   7.226114   
           AAP     41.161922     843400.0  39.806780  40.507213  40.935180   
           AAPL    27.833111  111727000.0  25.663924  25.941177  27.359656   
           ABC     24.367403    4210300.0  23.548246  23.958760  24.112215   

                         5MA     50STD     25STD     10STD      50UBB  ...  \
Date       Ticker                                                      ...   
2010-03-16 A       21.858573  0.961589  0.932832  0.303887  22.064953  ...   
           AAL      7.428802  0.970238  0.475839  0.240773   7.886535  ...   
           AAP     41.078227  1.027993  0.881267  0.261797  41.862767  ...   
           AAPL    27.907019  1.173133  1.326524  0.761078  28.010189  ...   
           ABC     24.087297  0.604464  0.298869  0.166858  24.757174  ...   

                   MACD5-25  MACD5-50  MACD10-25  MACD10-50  MACD25-50  \
Date       Ticker                                                        
2010-03-16 A       0.830135  1.242865   0.625629   1.038359   0.412730   
           AAL     0.574131  1.170651   0.457157   1.053676   0.596519   
           AAP     0.585189  0.970875   0.411384   0.797070   0.385686   
           AAPL    1.321352  1.690658   0.896776   1.266081   0.369306   
           ABC     0.178486  0.452716   0.154857   0.429087   0.274230   

                   5DMomentum  10DMomentum  25DMomentum  50DMomentum  target  
Date       Ticker                                                             
2010-03-16 A         0.097935     0.692101     2.755337          0.0       1  
           AAL       0.056563     0.084847     1.734644          0.0      -1  
           AAP       0.350330     0.934185     1.449917          0.0       1  
           AAPL      0.177330     1.934494     3.761101          0.0       0  
           ABC       0.317907    -0.085918     1.279705          0.0       1  

[5 rows x 28 columns]

In [21]:
from sklearn.preprocessing import StandardScaler

stock_data_indicators = stock_data_indicators.loc[(slice('2010-03-16','2019-12-31')),:]
indicators = stock_data_indicators.drop(columns=['Adj Close','Volume','target'])
indicators = indicators.values
indicators = StandardScaler().fit_transform(indicators)


In [22]:
from sklearn.decomposition import PCA
pca = PCA(.95)
principalComponents = pca.fit_transform(indicators)

In [23]:
indicator_index = stock_data_indicators.index

In [24]:
principalDf = pd.DataFrame(data = principalComponents, columns = ['PC1', 'PC2','PC3','PC4','PC5','PC6'], index = indicator_index)
principalDf.head()

PC1       PC2       PC3       PC4       PC5       PC6
Date       Ticker                                                            
2010-03-16 A      -0.752058 -1.622765  1.379154  0.829721  0.766182 -0.299314
           AAL    -1.155141 -1.345847  0.508184  0.746179  1.022953 -0.384040
           AAP    -0.637966 -0.966827  0.851218  0.315813  0.047650 -0.163081
           AAPL   -0.503010 -1.684244  1.363548  0.603027  0.750602 -0.149463
           ABC    -1.058094 -0.997907  0.891929  0.373091 -0.156564 -0.012736

In [26]:
stock_indicator_data = stock_data_indicators.loc[(slice('2010-03-16','2019-12-31')),:]

stock_indicator_data['PC1'] = principalDf['PC1']
stock_indicator_data['PC2'] = principalDf['PC2']
stock_indicator_data['PC3'] = principalDf['PC3']
stock_indicator_data['PC4'] = principalDf['PC4']
stock_indicator_data['PC5'] = principalDf['PC5']
stock_indicator_data['PC6'] = principalDf['PC6']

stock_indicator_data = stock_indicator_data.reindex(columns= ['Adj Close', 'Volume', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6','target'])

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [27]:
stock_indicator_data.head()
### need to scale adj close and volume

Adj Close       Volume       PC1       PC2       PC3  \
Date       Ticker                                                         
2010-03-16 A       21.820704    3285900.0 -0.752058 -1.622765  1.379154   
           AAL      7.325101    5915400.0 -1.155141 -1.345847  0.508184   
           AAP     41.161922     843400.0 -0.637966 -0.966827  0.851218   
           AAPL    27.833111  111727000.0 -0.503010 -1.684244  1.363548   
           ABC     24.367403    4210300.0 -1.058094 -0.997907  0.891929   

                        PC4       PC5       PC6  target  
Date       Ticker                                        
2010-03-16 A       0.829721  0.766182 -0.299314       1  
           AAL     0.746179  1.022953 -0.384040      -1  
           AAP     0.315813  0.047650 -0.163081       1  
           AAPL    0.603027  0.750602 -0.149463       0  
           ABC     0.373091 -0.156564 -0.012736       1

In [177]:
#stock_indicator_data.to_csv('stock_indicator_data.csv')

Adj Close    Volume       PC1       PC2       PC3  \
Date       Ticker                                                      
2010-03-16 A        0.005604  0.003822 -0.752058 -1.622765  1.379154   
           AAL      0.001880  0.006881 -1.155141 -1.345847  0.508184   
           AAP      0.010572  0.000981 -0.637966 -0.966827  0.851218   
           AAPL     0.007148  0.129969 -0.503010 -1.684244  1.363548   
           ABC      0.006258  0.004898 -1.058094 -0.997907  0.891929   

                        PC4       PC5       PC6  target  
Date       Ticker                                        
2010-03-16 A       0.829721  0.766182 -0.299314       1  
           AAL     0.746179  1.022953 -0.384040      -1  
           AAP     0.315813  0.047650 -0.163081       1  
           AAPL    0.603027  0.750602 -0.149463       0  
           ABC     0.373091 -0.156564 -0.012736       1

Adj Close    Volume       PC1       PC2       PC3  \
Date       Ticker                                                      
2010-03-16 A        0.005604  0.003822 -0.752058 -1.622765  1.379154   
           AAL      0.001880  0.006881 -1.155141 -1.345847  0.508184   
           AAP      0.010572  0.000981 -0.637966 -0.966827  0.851218   
           AAPL     0.007148  0.129969 -0.503010 -1.684244  1.363548   
           ABC      0.006258  0.004898 -1.058094 -0.997907  0.891929   

                        PC4       PC5       PC6  target  
Date       Ticker                                        
2010-03-16 A       0.829721  0.766182 -0.299314       1  
           AAL     0.746179  1.022953 -0.384040      -1  
           AAP     0.315813  0.047650 -0.163081       1  
           AAPL    0.603027  0.750602 -0.149463       0  
           ABC     0.373091 -0.156564 -0.012736       1

In [175]:
date_start = pd.Timestamp('2010-03-16')
date_end = pd.Timestamp('2020-01-01')
AAPL = stock_indicator_data.loc[(slice(str(date_start),str(date_end)), 'AAPL'), :]

indicator_date_index = AAPL.index.droplevel(1)
indicator_date_index = pd.DataFrame(indicator_date_index)
indicator_date_index.to_csv('indicator_date_index.csv', index=False)

In [29]:
date_start = pd.Timestamp('2019-01-01')
date_end = pd.Timestamp('2020-01-01')
AAPL = stock_indicator_data.loc[(slice(str(date_start),str(date_end)), 'AAPL'), :]

test_date_index = AAPL.index.droplevel(1)
test_date_index = pd.DataFrame(test_date_index)
test_date_index.to_csv('test_date_index.csv', index=False)

In [10]:
def get_rsi(window_length, data):
    df = data
    delta = df['Adj Close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up1 = up.ewm(span=window_length).mean()
    roll_down1 = down.abs().ewm(span=window_length).mean()
    RS = roll_up1 / roll_down1
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI

In [11]:
def get_macd(ewm1, ewm2, data):
    df = data
    exp1 = df['Adj Close'].ewm(span=ewm1, adjust=False).mean()
    exp2 =  df['Adj Close'].ewm(span=ewm2, adjust=False).mean()
    return exp1-exp2

In [13]:
def get_n_day_momentum(n_days, data):
    close = data['Adj Close']
    return close - close.shift(n_days)